In [3]:
# The code was removed by Watson Studio for sharing.

<H2>Data analysis project for Toronto</H2>
<p> We want to explore and cluster the neighborhoods in Toronto. </p>
<p> So let's get started:</p>

## Import the necessary packages

In [17]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import json
import requests

# Adding wikitables library for getting table in wikipedia
!pip install wikitables
import wikitables as wk

# Adding Geocoder library
!pip install geocoder
import geocoder

print('Packages have been added!')

Packages have been added!


<h1>First part</h1>

## Getting the data

Lets first get tables of the page using wikitables library. Using wikitables you can extract the table you need with few lines of codes instead of using api directly.

In [5]:
page_tables = wk.import_tables('List_of_postal_codes_of_Canada:_M') # Getting the page tables as an object
Toronto_table = page_tables[0].json() # Getting first table of the page and convert it to json
df = pd.read_json(Toronto_table) # Our dataframe
df = df[['Postcode', 'Borough', 'Neighbourhood']] # Just to sort them

As you Can see we have our table pretty easy and that's all you need!

In [6]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 1. Droping NOT assigned neighborhoods

In [7]:
df = df[df.Borough != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


## 2. Merging duplicate rows

In [8]:
# Merging Neighbourhoods for same post codes and getting combining them in a single row
df['Neighbourhood'] = df[['Postcode', 'Borough', 'Neighbourhood']].groupby(['Postcode'])['Neighbourhood'].transform(lambda x: ', '.join(x))

# Dropping duplicate rows
df = df.drop_duplicates()

In [9]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"
8,M7A,Queen's Park,Not assigned


## 3. Changing Not assigned Neighbourhoods to their borough name

In [10]:
df.loc[df.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df.Borough

In [11]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"
8,M7A,Queen's Park,Queen's Park


In [12]:
df.shape

(103, 3)

<h1>Second part</h1>

## 1. Getting the latitude and the longitude for each neighbourhood.

<font color="red"><b>After multiple tries Geocoder still gaved us None value's so as instructed, i used .csv file provided in the notebook. Still i wrote the method to get latitude and longitude just in case.</b></font>

In [38]:
latitude, longitude = [], []
for index, row in df.iterrows():
    
    # loop until you get the coordinates
    lat_lng_coords = None
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(str(row['Postcode'])))
      lat_lng_coords = g.latlng

    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    

Getting the csv data and making the columns of longitude and lantitude in our dataframe:

In [45]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Let's  add data to our dataframe.

In [54]:
for index, row in lat_lng_coords.iterrows():
    df.loc[df['Postcode'] == row['Postal Code'], 'Latitude'] = row['Latitude']
    df.loc[df['Postcode'] == row['Postal Code'], 'Longitude'] = row['Longitude']
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
6,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
8,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
